# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

In [4]:
import xml.etree.ElementTree as ET
import numpy as np
import re
from collections import Counter

### Procesamiento del Corpus:

- Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

In [7]:
# Cargar y analizar el contenido del archivo XML
arbol_xml = ET.parse('/content/sample_data/03ranking_corpus.xml')
raiz = arbol_xml.getroot()

# Crear una lista para almacenar la información de los textos y sus metadatos
lista_documentos = []

# Recorrer cada nodo 'documento' dentro del archivo XML
for nodo in raiz.findall('document'):
    identificador = nodo.get('id')
    titulo = nodo.find('title').text
    palabras_clave = nodo.find('keywords').text
    autor = nodo.find('author').text
    fecha = nodo.find('date').text

    # Añadir los metadatos del documento a la lista
    lista_documentos.append({
        'id': identificador,
        'titulo': titulo,
        'palabras_clave': palabras_clave,
        'autor': autor,
        'fecha': fecha
    })


In [8]:
# Mostrar la información de cada documento en la lista
for doc in lista_documentos:
    print(f"Identificador: {doc['id']}")
    print(f"Título: {doc['titulo']}")
    print(f"Palabras clave: {doc['palabras_clave']}")
    print(f"Autor: {doc['autor']}")
    print(f"Fecha: {doc['fecha']}")
    print("-" * 40)


Identificador: 1
Título: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
Palabras clave: telemedicina, salud crónica, tratamiento, tecnología médica
Autor: Dr. Juan Pérez
Fecha: 2023-01-15
----------------------------------------
Identificador: 2
Título: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
Palabras clave: nutrición, rendimiento académico, salud mental, estudiantes
Autor: Dra. María López
Fecha: 2023-02-10
----------------------------------------
Identificador: 3
Título: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
Palabras clave: amistad, bienestar estudiantil, campus, relaciones sociales
Autor: Miguel Rodríguez
Fecha: 2023-03-05
----------------------------------------
Identificador: 4
Título: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
Palabras clave: bibliotecas universitarias, investigación

### Procesamiento de las Consultas:

- Definir las consultas proporcionadas.
- Extraer las palabras clave de cada consulta.

In [9]:
# Definir las preguntas
preguntas = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

# Función para analizar el texto y extraer términos clave
def analizar_texto(texto):
    # Convertir el texto a minúsculas
    texto = texto.lower()
    # Sustituir caracteres no alfanuméricos por espacios
    import re
    texto = re.sub(r'[^a-záéíóúñü]+', ' ', texto)
    # Dividir en tokens y eliminar espacios adicionales
    tokens = texto.strip().split()
    return set(tokens)


In [10]:
# Mostrar los términos clave extraídos de cada pregunta
for id_pregunta, texto_pregunta in preguntas.items():
    # Analizar el texto de la pregunta
    palabras_clave = analizar_texto(texto_pregunta)
    print(f"Pregunta {id_pregunta}: {palabras_clave}")


Pregunta 1: {'la', 'de', 'estudiantes', 'universitarios', 'mental', 'impacto', 'en', 'el', 'salud', 'académico', 'los', 'rendimiento'}
Pregunta 2: {'emocional', 'extracurriculares', 'en', 'el', 'bienestar', 'campus', 'actividades', 'universitario', 'y'}
Pregunta 3: {'estrategias', 'estudiantes', 'estrés', 'en', 'el', 'universitarias', 'para', 'reducir'}


### Cálculo de Relevancia:
- Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.
- Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.

In [11]:
# Función para calcular la similitud coseno
def calcular_similitud_coseno(pregunta, documento):
    # Extraer términos clave de la pregunta y el documento
    tokens_pregunta = analizar_texto(pregunta)
    tokens_documento = analizar_texto(documento)

    # Generar el vocabulario combinado (palabras únicas de ambos conjuntos)
    vocabulario = list(tokens_pregunta | tokens_documento)  # Unión de los conjuntos

    # Construir los vectores binarios de frecuencia
    vector_pregunta = [1 if palabra in tokens_pregunta else 0 for palabra in vocabulario]
    vector_documento = [1 if palabra in tokens_documento else 0 for palabra in vocabulario]

    # Calcular el producto punto y las normas de los vectores
    producto_punto = np.dot(vector_pregunta, vector_documento)
    norma_pregunta = np.linalg.norm(vector_pregunta)
    norma_documento = np.linalg.norm(vector_documento)

    # Retornar la similitud coseno
    if norma_pregunta == 0 or norma_documento == 0:
        return 0  # No hay similitud si alguno de los vectores es cero
    return producto_punto / (norma_pregunta * norma_documento)


In [12]:
# Función para calcular la similitud de Jaccard
def calcular_similitud_jaccard(pregunta, documento):
    # Obtener los conjuntos de términos clave de la pregunta y el documento
    tokens_pregunta = analizar_texto(pregunta)
    tokens_documento = analizar_texto(documento)

    # Calcular la intersección y unión de los conjuntos
    interseccion = tokens_pregunta & tokens_documento
    union = tokens_pregunta | tokens_documento

    # Retornar la similitud de Jaccard
    return len(interseccion) / len(union) if union else 0


In [13]:
# Evaluar la relevancia de cada documento en relación con cada pregunta
for id_pregunta, texto_pregunta in preguntas.items():
    print(f"Pregunta {id_pregunta}: {texto_pregunta}")

    for documento in lista_documentos:
        texto_documento = documento['titulo'] + ' ' + documento['palabras_clave']

        # Calcular similitud coseno entre la pregunta y el documento
        puntuacion_coseno = calcular_similitud_coseno(texto_pregunta, texto_documento)

        # Calcular similitud de Jaccard entre la pregunta y el documento
        puntuacion_jaccard = calcular_similitud_jaccard(texto_pregunta, texto_documento)

        # Mostrar los resultados
        print(f"  Documento {documento['id']}: {documento['titulo']}")
        print(f"    Similitud Coseno: {puntuacion_coseno:.4f}")
        print(f"    Similitud de Jaccard: {puntuacion_jaccard:.4f}")
    print("\n")


Pregunta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
  Documento 1: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
    Similitud Coseno: 0.3203
    Similitud de Jaccard: 0.1905
  Documento 2: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
    Similitud Coseno: 0.6405
    Similitud de Jaccard: 0.4706
  Documento 3: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
    Similitud Coseno: 0.3501
    Similitud de Jaccard: 0.2083
  Documento 4: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
    Similitud Coseno: 0.3203
    Similitud de Jaccard: 0.1905
  Documento 5: Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés.
    Similitud Coseno: 0.3608
    Similitud de Jaccard: 0.2174
  Documento 6: La influencia de la cult

### Ranking de Documentos:

- Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
- Mostrar el ranking de documentos para cada consulta.

In [14]:
# Definir el número máximo de documentos a incluir en el ranking
limite_ranking = 7

# Diccionario para almacenar los rankings de relevancia de cada pregunta
rankings_relevancia = {}

# Iterar sobre cada pregunta para calcular similitudes y generar el ranking
for id_pregunta, texto_pregunta in preguntas.items():
    # Lista para almacenar los documentos junto con sus puntuaciones de relevancia
    puntuaciones_relevancia = []

    # Recorrer cada documento del corpus
    for documento in lista_documentos:
        # Crear el texto del documento concatenando título y palabras clave
        texto_documento = documento['titulo'] + ' ' + documento['palabras_clave']

        # Calcular similitud coseno y Jaccard utilizando las funciones definidas
        puntuacion_coseno = calcular_similitud_coseno(texto_pregunta, texto_documento)
        puntuacion_jaccard = calcular_similitud_jaccard(texto_pregunta, texto_documento)

        # Calcular la puntuación combinada de relevancia
        puntuacion_relevancia = (puntuacion_coseno + puntuacion_jaccard) / 2

        # Agregar el documento con sus puntuaciones a la lista
        puntuaciones_relevancia.append({
            'id_documento': documento['id'],
            'titulo': documento['titulo'],
            'puntuacion_coseno': puntuacion_coseno,
            'puntuacion_jaccard': puntuacion_jaccard,
            'puntuacion_relevancia': puntuacion_relevancia
        })

    # Ordenar los documentos por la puntuación de relevancia en orden descendente
    puntuaciones_relevancia = sorted(puntuaciones_relevancia, key=lambda x: x['puntuacion_relevancia'], reverse=True)

    # Guardar solo los `k` documentos más relevantes para la pregunta actual
    rankings_relevancia[id_pregunta] = puntuaciones_relevancia[:limite_ranking]


In [15]:
# Mostrar el ranking de documentos para cada pregunta
for id_pregunta, documentos_ordenados in rankings_relevancia.items():
    print(f"\nRanking para la Pregunta {id_pregunta}: {preguntas[id_pregunta]}")
    for posicion, documento in enumerate(documentos_ordenados, start=1):
        print(f"  {posicion}. Documento {documento['id_documento']}: {documento['titulo']}")
        print(f"     Similitud Coseno: {documento['puntuacion_coseno']:.4f}")
        print(f"     Similitud de Jaccard: {documento['puntuacion_jaccard']:.4f}")
        print(f"     Puntuación Total de Relevancia: {documento['puntuacion_relevancia']:.4f}")



Ranking para la Pregunta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
  1. Documento 2: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
     Similitud Coseno: 0.6405
     Similitud de Jaccard: 0.4706
     Puntuación Total de Relevancia: 0.5556
  2. Documento 7: La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
     Similitud Coseno: 0.6405
     Similitud de Jaccard: 0.4706
     Puntuación Total de Relevancia: 0.5556
  3. Documento 14: Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
     Similitud Coseno: 0.6301
     Similitud de Jaccard: 0.4500
     Puntuación Total de Relevancia: 0.5401
  4. Documento 11: Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.
     Similitud Coseno: 0.6172
     Similitud de Jaccard: 0.4444
     Puntuación 